# Loading train data 

<h5 dir="rtl">
در این بخش مدل زبانی N-gram را برای انجام این تسک استفاده می‌کنیم. ابتدا برای آموزش مدل، tokenization و normalization را روی متن اعمال می‌کنیم. 
<h5\>

In [1]:
!pip install hazm
!pip install stanza
!pip install spacy
!pip install spacy_stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def pbar(x):
  return tqdm(x, position=0, leave=True)

In [5]:
import codecs
from tqdm import tqdm

lines = [x for i, x in pbar(enumerate(codecs.open('/content/drive/MyDrive/Colab Notebooks/NLP/sports.txt','rU','utf-8').readlines())) if i<820000]
paragraphs = []
paragraph = ''
for line in pbar(lines):
    if line == '\n':
        paragraphs.append(paragraph)
        paragraph = ''
    else:
        paragraph += line
else:
    paragraphs.append(paragraph)

1833371it [00:00, 2041135.67it/s]
100%|██████████| 820000/820000 [00:00<00:00, 1480361.28it/s]


In [6]:
from hazm import Normalizer
from hazm import sent_tokenize
from itertools import chain
normalizer = Normalizer()
paragraphs_normalized = [normalizer.normalize(paragraph) for paragraph in pbar(paragraphs)]
sentences = list(chain(*[sent_tokenize(paragraph) for paragraph in pbar(paragraphs_normalized)]))

100%|██████████| 92195/92195 [00:07<00:00, 11815.32it/s]


# Language Model

<h5 dir="rtl">
مدل به کار رفته همان مدل ارايه شده در کلاس است. صرفا تغییراتی روی آن اعمال شده. برای استخراج توکن ها به جای تابع split از nltk.tokenizer استفاده کردیم که مدل را بهبود داد. در مراحل میانی متغیرهای حجیمی را که در ادامه استفاده نمی شدند حذف کردیم تا در مصرف RAM صرفه‌جویی شود. تغیر مهم دیگر این بود که مدل اگر یک دنباله از کلمات را در مدل k-gram پیدا نکند، به دنبال  کلمات در مدل k-1-gram می‌گردد و ...  تابع best_candidate نیز در مواردی دچار خطا می‌شد که ایرادات آن نیز رفع شد.
<h5\>

In [7]:
import math
import nltk
import pickle
import os
import numpy as np
from itertools import product
from os.path import exists
import gc
from hazm import word_tokenize

class LanguageModel(object):
    """An n-gram language model trained on a given corpus.
    
    For a given n and given training corpus, constructs an n-gram language
    model for the corpus by:
    1. preprocessing the corpus (adding SOS/EOS/UNK tokens)
    2. calculating (smoothed) probabilities for each n-gram
    Also contains methods for calculating the perplexity of the model
    against another corpus, and for generating sentences.
    Args:
        train_data (list of str): list of sentences comprising the training corpus.
        n (int): the order of language model to build (i.e. 1 for unigram, 2 for bigram, etc.).
        laplace (int): lambda multiplier to use for laplace smoothing (default 1 for add-1 smoothing).
    """

    SOS = "<s>"
    EOS = "</s>"
    UNK = "<UNK>"
    
    def __init__(self, train_data, n, laplace=1, recalculate=True):
        self.n = n
        self.recalculate = recalculate
        self.vocab = dict()
        self.laplace = laplace
        self.tokens = None
        self.models = None

        self.preprocess(train_data, n)
        self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        """Apply Laplace smoothing to n-gram frequency distribution.
        
        Here, n_grams refers to the n-grams of the tokens in the training corpus,
        while m_grams refers to the first (n-1) tokens of each n-gram.
        Returns:
            dict: Mapping of each n-gram (tuple of str) to its Laplace-smoothed 
            probability (float).
        """
        pickle_name = f'models_{str(self.n)}.pickle'
        if self.recalculate or not os.path.exists(pickle_name):
            print('Recalculating model')
            vocab_size = len(self.vocab)
            n_vocabs = [None]*(self.n+1)
            for i in range(2, self.n+1):
                n_grams = nltk.ngrams(self.tokens, i)
                n_vocab = nltk.FreqDist(n_grams)
                n_vocabs[i] = n_vocab
                del(n_grams)
            gc.collect()
            models = [None]*(self.n+1)
            def smoothed_count(n_gram, n_count, n):
                m_gram = n_gram[:-1]
                m_count = n_vocabs[i][m_gram]
                return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

            num_tokens = len(self.tokens)
            models[1] = { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
            for i in range(2, self.n+1):
                model = {
                    n_gram: smoothed_count(n_gram, count, i) for n_gram, count in n_vocabs[i].items()
                }
                models[i] = model
            self.models = models
            pickle_out = open(pickle_name,"wb")
            pickle.dump(models, pickle_out)
            pickle_out.close()
            del(n_vocabs)
            gc.collect()
        else:
            print('Loading model')
            pickle_in = open(pickle_name,"rb")
            self.models = pickle.load(pickle_in)


    def _create_model(self):
        """Create a probability distribution for the vocabulary of the training corpus.
        
        If building a unigram model, the probabilities are simple relative frequencies
        of each token with the entire corpus.
        Otherwise, the probabilities are Laplace-smoothed relative frequencies.
        Returns:
            A dict mapping each n-gram (tuple of str) to its probability (float).
        """
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        """Convert, if necessary, a given n-gram to one which is known by the model.
        Starting with the unmodified ngram, check each possible permutation of the n-gram
        with each index of the n-gram containing either the original token or <UNK>. Stop
        when the model contains an entry for that permutation.
        This is achieved by creating a 'bitmask' for the n-gram tuple, and swapping out
        each flagged token for <UNK>. Thus, in the worst case, this function checks 2^n
        possible n-grams before returning.
        Returns:
            The n-gram with <UNK> tokens in certain positions such that the model
            contains an entry for it.
        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        """Calculate the perplexity of the model against a given test corpus.
        
        Args:
            test_data (list of str): sentences comprising the training corpus.
        Returns:
            The perplexity of the model as a float.
        
        """
        test_tokens = self.preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = [self._convert_oov(ngram) for ngram in test_ngrams]
        probabilities = [self.model[ngram] for ngram in known_ngrams]
        
        for x,y in zip(known_ngrams, probabilities):
            print(x,y)
        
        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, without=[], random=False):
        
        blacklist  = [LanguageModel.UNK] + without

        if len(prev) < self.n-1:
            prev = [LanguageModel.SOS]*(self.n-1 - len(prev)) + prev
        elif len(prev) > self.n-1:
            prev = prev[-(self.n-1):]
        candidates = []
        n = self.n
        while len(candidates) == 0 and n > 0:
            candidates = list(((ngram[-1],prob) for ngram,prob in self.models[n].items() if ngram[:-1]==tuple(prev[-(n-1):])))
            n -= 1

        if n == 0:
            return [("",0)]
        return sorted(candidates, key=lambda x: -x[1])

    def generate_sentence(self, min_len=12, max_len=24):
        sent, prob = ([LanguageModel.SOS] * (max(1, self.n-1)), 1)
        while sent[-1] != LanguageModel.EOS:
            prev = [] if self.n == 1 else sent[-(self.n-1):]
            blacklist = sent + ([LanguageModel.EOS,LanguageModel.SOS] if len(sent) < min_len else [])
            next_token, next_prob = self._best_candidate(prev, without=blacklist)
            sent.append(next_token)
            prob *= next_prob

            if len(sent) >= max_len:
                sent.append(LanguageModel.EOS)

        return (' '.join(sent[(self.n-1):-1]), -1/math.log(prob))
    
    

    def add_sentence_tokens(self, sentences, n):
        """Wrap each sentence in SOS and EOS tokens.
        For n >= 2, n-1 SOS tokens are added, otherwise only one is added.
        Args:
            sentences (list of str): the sentences to wrap.
            n (int): order of the n-gram model which will use these sentences.
        Returns:
            List of sentences with SOS and EOS tokens wrapped around them.
        """
        sos = ' '.join([LanguageModel.SOS] * (n-1)) if n > 1 else LanguageModel.SOS
        return ['{} {} {}'.format(sos, s, LanguageModel.EOS) for s in sentences]

    def replace_singletons(self, tokens):
        """Replace tokens which appear only once in the corpus with <UNK>.

        Args:
            tokens (list of str): the tokens comprising the corpus.
        Returns:
            The same list of tokens with each singleton replaced by <UNK>.

        """
        if len(self.vocab) == 0:
            self.vocab = nltk.FreqDist(tokens)
        return [token if self.vocab[token] > 1 else LanguageModel.UNK for token in tokens]

    def preprocess(self, sentences, n):
        """Add SOS/EOS/UNK tokens to given sentences and tokenize.
        Args:
            sentences (list of str): the sentences to preprocess.
            n (int): order of the n-gram model which will use these sentences.
        Returns:
            The preprocessed sentences, tokenized by words.
        """
        pickle_name = f'tokens_{str(self.n)}.pickle'

        if self.recalculate or not os.path.exists(pickle_name):
            print('Recalculating tokens')
            sentences = self.add_sentence_tokens(sentences, n)
            tokens =word_tokenize(' '.join(sentences))
            self.tokens = self.replace_singletons(tokens)
            pickle_out = open(pickle_name,"wb")
            pickle.dump(tokens, pickle_out)
            pickle_out.close()
        else:
            print('Loading tokens')
            pickle_in = open(pickle_name,"rb")
            self.tokens = pickle.load(pickle_in)

        pickle_name = f'vocab_{str(self.n)}.pickle'
        if self.recalculate or not exists(pickle_name):
            print('Recalculating vocab')
            self.vocab  = nltk.FreqDist(self.tokens)
            pickle_out = open(pickle_name,"wb")
            pickle.dump(self.vocab, pickle_out)
            pickle_out.close()
        else:
            print('Loading vocab')
            pickle_in = open(pickle_name,"rb")
            self.vocab = pickle.load(pickle_in)
          

In [8]:
language_model = LanguageModel(sentences, 3, 1, recalculate=True)

Recalculating tokens
Recalculating vocab
Recalculating model


# Test

<h5 dir="rtl">
حال با استفاده از تابع best_candidate می‌خواهیم غلط های موجود را تصحیح کنیم. ابتدا متن ورودی را نرمال می‌کنیم. سپس به ازای هر توکن از متن، کلمات قبل از توکن را به ترتیب به تابع می‌دهیم تا بهترین حدس های مدل را خروجی دهد. سپس کلماتی که احتمال وقوع آن‌ها از یک مقدار آستانه کمتر باشد را حذف میکنیم. در مرحله‌ی بعدی کلماتی که نسبت فاصله‌ی ویرایشی آن‌ها با کلمه‌ی ورودی تقسیم بر طول کلمه‌ی ورودی از یک مقدار آستانه بیشتر باشد را حذف میکنیم. به زبان ساده برای کلمه‌های ۳و۴ حرفی اجازه‌ی یک فاصله‌ی نگارشی و برای کلمات ۴و۵و۶ و۷ حرفی اجازه‌ی دو فاصله‌ی نگارشی را می‌دهیم. در این بین اگر کلمه‌ی ورودی یکی از حروف اضافه باشد پردازشی روی آن رخ نمی دهد. همچنین اگر هیچ کلمه‌ای از فیلتر های فوق رد نشود نیز تغییری در کلمه‌ی ورودی داده نمی‌شود. در غیر این صورت کلمه‌ای که بیشنه‌ی معیار گفته شده را داشته باشد به عنوان بهترین کاندید خروجی می دهیم.  توجه کنید که به دلیل محدودیت RAM ما مدل را فقط بر روی بخشی از داده‌ی فایل sport.txt آموزش دادیم. لذا این مدل قادر به تصحیح غلط های املایی متن‌هایی با همین موضوع خواهد بود. طبیعتا با داده ی بیشتر عملکرد مدل بهتر می‌شود اما به دلیل محدودیت سخت‌افزاری قادر به آموزش مدل با داده ي بیشتر نبودیم.
برای مثال جمله ی "بازی برگذار شد." را در نظر بگیرید ابتدا  "بازی برگذار" به تابع best_candidate داده می‌شود که چون "برگذار" از نظر املایی غلط است و کلمه‌ی جایگزین "برگزار" معیارهای گفته شده را دارد پس این تصحیح صورت می‌گیرد. سپس "بازی برگذار شد" به تابع داده می‌شود که جایگزینی برای "شد" پیشنهاد نمی‌شود و همین طور بعد از دادن "بازی برگذار شد." جایگزین بهتری برای "." پیدا نمی‌شود.
<h5/>

In [9]:
from spacy.tokens.token import Token
import editdistance
import pandas as pd
import stanza
import spacy_stanza

stanza.install_corenlp()
stanza.download('fa')
nlp = spacy_stanza.load_pipeline("fa")

2022-06-06 17:00:11 WARNING: Directory /root/stanza_corenlp already exists. Please install CoreNLP to a new directory.


2022-06-06 17:00:11 INFO: Downloading default packages for language: fa (Persian)...
2022-06-06 17:00:14 INFO: File exists: /root/stanza_resources/fa/default.zip
2022-06-06 17:00:18 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-06-06 17:00:19 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |
| ner       | arman   |

2022-06-06 17:00:19 INFO: Use device: cpu
2022-06-06 17:00:19 INFO: Loading: tokenize
2022-06-06 17:00:19 INFO: Loading: mwt
2022-06-06 17:00:19 INFO: Loading: pos
2022-06-06 17:00:20 INFO: Loading: lemma
2022-06-06 17:00:20 INFO: Loading: depparse
2022-06-06 17:00:20 INFO: Loading: ner
2022-06-06 17:00:21 INFO: Done loading processors!


In [12]:

texes = [
"امروز دوشنبه به مساف تیم ملی فوتبال رفت."
, "تیم حندباز ایتالیا قهرمان شد."
, "فدراسیون بانون با این تسمیم موافقت کرد."
, "پیوستن مسی به بارصلونا تاریخی بود."
, "محمد بننا به اذربایجان صفر کرد."
, "به توپ زربه زد."
, "بازی را واگذار کرد."
, "بازی را به تصاوی کشاند."
, "کمیته فن این فدراسیون در شرایطی بعد از المپیک جلسات مخطلفی را برگذار کرد."
, "مسابقه امروز برگزار می‌شود و هریفان به مساف هم می‌روند."
, "مسابقه‌ی فوتبال با تاخیر برگذار شد."
]

stop_words=['به','که','از','در','با','برای','تا','را']

TRESHOLD = 0.4
MIN_SCORE = 0.000001

for text in texes:
  k =  len(nlp(text))
  text = normalizer.normalize(text)

  for index in range(1,k-1):

      doc = nlp(text)
      current_token: Token = doc[index]
      start_char_index = current_token.idx
      end_char_index = start_char_index + len(current_token)
      current_text = list(map(lambda x: x.text,doc[:index]))

      if current_token.text in stop_words:
        continue

      predicts = language_model._best_candidate(current_text)
      if predicts[0][0] is None:
        continue

      predicts = pd.DataFrame(predicts, columns = ['candidate', 'probability'])
      predicts = predicts[predicts['probability']>MIN_SCORE]
      predicts['edit_distance'] = predicts['candidate'].apply(lambda tk:editdistance.eval(current_token.text.strip(), tk.strip())/len(tk.strip()))
      predicts = predicts[predicts['edit_distance']<=0.4]
      predicts = predicts.sort_values(by="edit_distance", ascending=True)

      if predicts['candidate'].empty:
        continue

      selected_predict = predicts['candidate'].iloc[0]
      if selected_predict.strip()!=current_token.text.strip():
        output = {"raw":current_token.text,"corrected":selected_predict,"span":[start_char_index,end_char_index]}
        print(output)


{'raw': 'دوشنبه', 'corrected': 'یکشنبه', 'span': [6, 12]}
{'raw': 'مساف', 'corrected': 'مصاف', 'span': [16, 20]}
{'raw': 'حندباز', 'corrected': 'هندبال', 'span': [4, 10]}
{'raw': 'بانون', 'corrected': 'کانوی', 'span': [9, 14]}
{'raw': 'تسمیم', 'corrected': 'تصمیم', 'span': [22, 27]}
{'raw': 'بارصلونا', 'corrected': 'بارسلونا', 'span': [14, 22]}
{'raw': 'بننا', 'corrected': 'بنا', 'span': [5, 9]}
{'raw': 'اذربایجان', 'corrected': 'آذربایجان', 'span': [13, 22]}
{'raw': 'زربه', 'corrected': 'ضربه', 'span': [7, 11]}
{'raw': 'تصاوی', 'corrected': 'تساوی', 'span': [11, 16]}
{'raw': 'فن', 'corrected': 'فنی', 'span': [6, 8]}
{'raw': 'مخطلفی', 'corrected': 'مختلفی', 'span': [52, 58]}
{'raw': 'برگذار', 'corrected': 'برگزار', 'span': [62, 68]}
{'raw': 'هریفان', 'corrected': 'حریفان', 'span': [29, 35]}
{'raw': 'مساف', 'corrected': 'مسافر', 'span': [39, 43]}
{'raw': 'برگذار', 'corrected': 'برگزار', 'span': [25, 31]}
